#Open Library Analysis - Big Data Computing Project
####Graph's Connected Components vs. k-means Clusters

##Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

##Data Aquisition

short dataset: 50000 entries

In [0]:
%sh wget -P /tmp https://raw.githubusercontent.com/attennig/BDC_datasets/main/books_short.csv

--2021-06-22 23:33:44-- https://raw.githubusercontent.com/attennig/BDC_datasets/main/books_short.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8067575 (7.7M) [text/plain]
Saving to: ‘/tmp/books_short.csv’

 0K .......... .......... .......... .......... .......... 0% 6.36M 1s
 50K .......... .......... .......... .......... .......... 1% 5.24M 1s
 100K .......... .......... .......... .......... .......... 1% 5.88M 1s
 150K .......... .......... .......... .......... .......... 2% 6.53M 1s
 200K .......... .......... .......... .......... .......... 3% 5.42M 1s
 250K .......... .......... .......... .......... .......... 3% 41.7M 1s
 300K .......... .......... .......... .......... .......... 4% 9.18M 1s
 350K .......... .......... .......... .......... .......... 5% 55.9M 1s
 400K .......... .......... .......... .......... .......... 5% 37.6M 1s
 450K .......... .......... .......... .......... .......... 6% 38.8M 1s
 500K .......... .......... .......... .......... .......... 6% 19.0M 1s
 550K .......... .......... .......... .......... .......... 7% 19.0M 1s
 600K .......... .......... .......... .......... .......... 8% 75.4M 1s
 650K .......... .......... .......... .......... .......... 8% 99.8M 1s
 700K .......... .......... .......... .......... .......... 9% 78.3M 1s
 750K .......... .......... .......... .......... .......... 10% 117M 1s
 800K .......... .......... .......... .......... .......... 10% 98.8M 0s
 850K .......... .......... .......... .......... .......... 11% 20.9M 0s
 900K .......... .......... .......... .......... .......... 12% 101M 0s
 950K .......... .......... .......... .......... .......... 12% 116M 0s
 1000K .......... .......... .......... .......... .......... 13% 25.0M 0s
 1050K .......... .......... .......... .......... .......... 13% 108M 0s
 1100K .......... .......... .......... .......... .......... 14% 93.0M 0s
 1150K .......... .......... .......... .......... .......... 15% 72.7M 0s
 1200K .......... .......... .......... .......... .......... 15% 73.8M 0s
 1250K .......... .......... .......... .......... .......... 16% 70.8M 0s
 1300K .......... .......... .......... .......... .......... 17% 65.3M 0s
 1350K .......... .......... .......... .......... .......... 17% 115M 0s
 1400K .......... .......... .......... .......... .......... 18% 48.4M 0s
 1450K .......... .......... .......... .......... .......... 19% 89.3M 0s
 1500K .......... .......... .......... .......... .......... 19% 23.9M 0s
 1550K .......... .......... .......... .......... .......... 20% 113M 0s
 1600K .......... .......... .......... .......... .......... 20% 80.7M 0s
 1650K .......... .......... .......... .......... .......... 21% 85.8M 0s
 1700K .......... .......... .......... .......... .......... 22% 63.1M 0s
 1750K .......... .......... .......... .......... .......... 22% 66.2M 0s
 1800K .......... .......... .......... .......... .......... 23% 123M 0s
 1850K .......... .......... .......... .......... .......... 24% 77.3M 0s
 1900K .......... .......... .......... .......... .......... 24% 50.7M 0s
 1950K .......... .......... .......... .......... .......... 25% 68.1M 0s
 2000K .......... .......... .......... .......... .......... 26% 105M 0s
 2050K .......... .......... .......... .......... .......... 26% 23.9M 0s
 2100K .......... .......... .......... .......... .......... 27% 114M 0s
 2150K .......... .......... .......... .......... .......... 27% 72.7M 0s
 2200K .......... .......... .......... .......... .......... 28% 89.4M 0s
 2250K .......... .......... .......... .......... .......... 29% 100M 0s
 2300K .......... .......... .......... .......... .......... 29% 86.9M 0s
 2350K .......... .......... .......... .......... .......... 30% 66.7M 0s
 240

In [0]:
dbutils.fs.mv("file:/tmp/books_short.csv", "dbfs:/bdc-2020-21/datasets/books_short.csv")

Out[76]: True

long dataset: 22.589.356 entries 

remember to download it only if it is necessary

In [0]:
%sh wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1qIhBSrpkDc-RCdbw7e1NVtNhOj_fNi5G" -O /tmp/books_long.csv.bz2 && rm -rf /tmp/cookies.txt

In [0]:
dbutils.fs.ls("file:/tmp")

In [0]:
dbutils.fs.mv("file:/tmp/books_long.csv.bz2", "dbfs:/bdc-2020-21/datasets/books_long.csv.bz2")

In [0]:
dbutils.fs.ls("dbfs:/bdc-2020-21/datasets")

Out[77]: [FileInfo(path='dbfs:/bdc-2020-21/datasets/books_long.csv.bz2', name='books_long.csv.bz2', size=975607032),
 FileInfo(path='dbfs:/bdc-2020-21/datasets/books_short.csv', name='books_short.csv', size=8067575)]

In [0]:
# Read dataset file into a Spark Dataframe
books_df = spark.read.load("dbfs:/bdc-2020-21/datasets/books_short.csv", 
                         format="csv", 
                         sep=";", 
                         inferSchema="true", 
                         header="true"
                         )

In [0]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(books_df.count(), len(books_df.columns)))

The shape of the dataset is 50000 rows by 17 columns

##Data Cleaning

In [0]:
#eliminating entries with missing values or columns
columns_to_drop = []
for c in books_df.columns:
  if books_df.where(col(c).isNull()).count()/books_df.count() > 0.7:
    #remove col
    columns_to_drop += [c]

In [0]:
books_df = books_df.drop(*columns_to_drop)

In [0]:
assert "title" in books_df.columns and "key" in books_df.columns
books_df = books_df.dropna(how="any", subset=["key", "title"])
books_df = books_df.dropDuplicates(['key'])
books_df = books_df.dropDuplicates(['title'])

In [0]:
assert "subjects" in books_df.columns and "authors" in books_df.columns
books_df = books_df.na.fill({'subjects': 'unknown', 'authors': 'unknown'})

In [0]:
books_df = books_df.orderBy(["key"], ascending=[False, False])

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
# This will return a new DF with all the columns + id
books_df = books_df.withColumn("id", monotonically_increasing_id())

In [0]:
# if nltk not found
!python -m pip install nltk

!python -m pip install --upgrade pip

!python -m nltk.downloader all

Collecting nltk
 Downloading nltk-3.6.2-py3-none-any.whl (1.5 MB)
▎ | 10 kB 11.2 MB/s eta 0:00:01 |▌ | 20 kB 11.5 MB/s eta 0:00:01 |▊ | 30 kB 11.2 MB/s eta 0:00:01 |█ | 40 kB 8.5 MB/s eta 0:00:01 |█▏ | 51 kB 8.9 MB/s eta 0:00:01 |█▍ | 61 kB 6.5 MB/s eta 0:00:01 |█▋ | 71 kB 6.6 MB/s eta 0:00:01 |█▉ | 81 kB 6.0 MB/s eta 0:00:01 |██ | 92 kB 6.3 MB/s eta 0:00:01 |██▎ | 102 kB 5.9 MB/s eta 0:00:01 |██▌ | 112 kB 5.9 MB/s eta 0:00:01 |██▊ | 122 kB 5.9 MB/s eta 0:00:01 |███ | 133 kB 5.9 MB/s eta 0:00:01 |███▏ | 143 kB 5.9 MB/s eta 0:00:01 |███▍ | 153 kB 5.9 MB/s eta 0:00:01 |███▋ | 163 kB 5.9 MB/s eta 0:00:01 |███▉ | 174 kB 5.9 MB/s eta 0:00:01 |████ | 184 kB 5.9 MB/s eta 0:00:01 |████▎ | 194 kB 5.9 MB/s eta 0:00:01 |████▌ | 204 kB 5.9 MB/s eta 0:00:01 |████▊ | 215 kB 5.9 MB/s eta 0:00:01 |█████ | 225 kB 5.9 MB/s eta 0:00:01 |█████▏ | 235 kB 5.9 MB/s eta 0:00:01 |█████▍ | 245 kB 5.9 MB/s eta 0:00:01 |█████▋ | 256 kB 5.9 MB/s eta 0:00:01 |█████▉ | 266 kB 5.9 MB/s eta 0:00:01 |██████ | 276 kB 5.9 MB/s eta 0:00:01 |██████▎ | 286 kB 5.9 MB/s eta 0:00:01 |██████▌ | 296 kB 5.9 MB/s eta 0:00:01 |██████▊ | 307 kB 5.9 MB/s eta 0:00:01 |███████ | 317 kB 5.9 MB/s eta 0:00:01 |███████▏ | 327 kB 5.9 MB/s eta 0:00:01 |███████▍ | 337 kB 5.9 MB/s eta 0:00:01 |███████▊ | 348 kB 5.9 MB/s eta 0:00:01 |████████ | 358 kB 5.9 MB/s eta 0:00:01 |████████▏ | 368 kB 5.9 MB/s eta 0:00:01 |████████▍ | 378 kB 5.9 MB/s eta 0:00:01 |████████▋ | 389 kB 5.9 MB/s eta 0:00:01 |████████▉ | 399 kB 5.9 MB/s eta 0:00:01 |█████████ | 409 kB 5.9 MB/s eta 0:00:01 |█████████▎ | 419 kB 5.9 MB/s eta 0:00:01 |█████████▌ | 430 kB 5.9 MB/s eta 0:00:01 |█████████▊ | 440 kB 5.9 MB/s eta 0:00:01 |██████████ | 450 kB 5.9 MB/s eta 0:00:01 |██████████▏ | 460 kB 5.9 MB/s eta 0:00:01 |██████████▍ | 471 kB 5.9 MB/s eta 0:00:01 |██████████▋ | 481 kB 5.9 MB/s eta 0:00:01 |██████████▉ | 491 kB 5.9 MB/s eta 0:00:01 |███████████ | 501 kB 5.9 MB/s eta 0:00:01 |███████████▎ | 512 kB 5.9 MB/s eta 0:00:01 |███████████▌ | 522 kB 5.9 MB/s eta 0:00:01 |███████████▊ | 532 kB 5.9 MB/s eta 0:00:01 |████████████ | 542 kB 5.9 MB/s eta 0:00:01 |████████████▏ | 552 kB 5.9 MB/s eta 0:00:01 |████████████▍ | 563 kB 5.9 MB/s eta 0:00:01 |████████████▋ | 573 kB 5.9 MB/s eta 0:00:01 |████████████▉ | 583 kB 5.9 MB/s eta 0:00:01 |█████████████ | 593 kB 5.9 MB/s eta 0:00:01 |█████████████▎ | 604 kB 5.9 MB/s eta 0:00:01 |█████████████▌ | 614 kB 5.9 MB/s eta 0:00:01 |█████████████▊ | 624 kB 5.9 MB/s eta 0:00:01 |██████████████ | 634 kB 5.9 MB/s eta 0:00:01 |██████████████▏ | 645 kB 5.9 MB/s eta 0:00:01 |██████████████▍ | 655 kB 5.9 MB/s eta 0:00:01 |██████████████▋ | 665 kB 5.9 MB/s eta 0:00:01 |██████████████▉ | 675 kB 5.9 MB/s eta 0:00:01 |███████████████▏ | 686 kB 5.9 MB/s eta 0:00:01 |███████████████▍ | 696 kB 5.9 MB/s eta 0:00:01 |███████████████▋ | 706 kB 5.9 MB/s eta 0:00:01 |███████████████▉ | 716 kB 5.9 MB/s eta 0:00:01 |████████████████ | 727 kB 5.9 MB/s eta 0:00:01 |████████████████▎ | 737 kB 5.9 MB/s eta 0:00:01 |████████████████▌ | 747 kB 5.9 MB/s eta 0:00:01 |████████████████▊ | 757 kB 5.9 MB/s eta 0:00:01 |█████████████████ | 768 kB 5.9 MB/s eta 0:00:01 |█████████████████▏ | 778 kB 5.9 MB/s eta 0:00:01 |█████████████████▍ | 788 kB 5.9 MB/s eta 0:00:01 |█████████████████▋ | 798 kB 5.9 MB/s eta 0:00:01 |█████████████████▉ | 808 kB 5.9 MB/s eta 0:00:01 |██████████████████ | 819 kB 5.9 MB/s eta 0:00:01 |██████████████████▎ | 829 kB 5.9 MB/s eta 0:00:01 |██████████████████▌ | 839 kB 5.9 MB/s eta 0:00:01 |██████████████████▊ | 849 kB 5.9 MB/s eta 0:00:01 |███████████████████ | 860 kB 5.9 MB/s eta 0:00:01 |███████████████████▏ | 870 kB 5.9 MB/s eta 0:00:01 |███████████████████▍ | 880 kB 5.9 MB/s eta 0:00:01 |███████████████████▋ | 890 kB 5.9 MB/s eta 0:00:01 |███████████████████▉ | 901 kB 5.9 MB/s eta 0:00:01 |████████████████████ | 911 kB 5.9 MB/s eta 0:00:01 |████████████████████▎ | 921 kB 5.9 MB/s eta 0:00:01 |████████

In [0]:
# From Document_Clustering.ipynb
def clean_text(df, column_name, perform_stemming=True):
    """ 
    This function takes the raw text data and applies a standard NLP preprocessing pipeline consisting of the following steps:
      - Text cleaning
      - Tokenization
      - Stopwords removal
      - Stemming (Snowball stemmer)

    parameter: dataframe
    returns: the input dataframe along with the `cleaned_content` column as the results of the NLP preprocessing pipeline
    """
    from pyspark.sql.functions import udf, col, lower, trim, regexp_replace
    from pyspark.ml.feature import Tokenizer, StopWordsRemover
    from nltk.stem.snowball import SnowballStemmer # BE SURE NLTK IS INSTALLED ON THE CLUSTER USING THE "LIBRARIES" TAB IN THE MENU

    
    # Text preprocessing pipeline
    # 1. Text cleaning
    # 1.a Case normalization
    lower_case_df = df.select(["id",lower(col(column_name)).alias(column_name)])
    # 1.b Trimming
    trimmed_df = lower_case_df.select(["id",trim(col(column_name)).alias(column_name)])
    # 1.c Filter out punctuation symbols
    no_punct_df = trimmed_df.select(["id",(regexp_replace(col(column_name), "[^a-zA-Z\\s]", "")).alias(column_name)])
    # 1.d Filter out any internal extra whitespace
    cleaned_df = no_punct_df.select(["id",trim(regexp_replace(col(column_name), " +", " ")).alias(column_name)])
    # 2. Tokenization (split text into tokens)
    tokenizer = Tokenizer(inputCol=column_name, outputCol="tokens")
    tokens_df = tokenizer.transform(cleaned_df).cache()
    # 3. Stopwords removal
    stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="terms")
    ret_df = stopwords_remover.transform(tokens_df).cache()
    # 4. Stemming (Snowball stemmer)
    if perform_stemming:
      stemmer = SnowballStemmer(language="english")
      stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
      ret_df = ret_df.withColumn("terms_stemmed", stemmer_udf("terms")).cache()
      
    return ret_df

In [0]:
clean_title_df = clean_text(books_df, "title")

In [0]:
clean_subjects_df = clean_text(books_df, "subjects")

In [0]:
clean_authors_df = clean_text(books_df, "authors", False)

##Feature Engineering

In this section we will use nlp techniques to get numerical vectors that represent text-based features

In [0]:
# final df
# |id|w2vec(clean_title)|w2vec(clean_subject)|authors|

In [0]:
RANDOM_SEED = 42 # used below to run the actual K-means clustering
EMBEDDING_SIZE = 150 # size of embedding Word2Vec vectors

In [0]:
#Word2Vec from Document_Clustering.ipynb 
def extract_w2v_features(df, column_name, out_col_name):
  from pyspark.ml.feature import Word2Vec
  
  word2vec = Word2Vec(vectorSize=EMBEDDING_SIZE, minCount=5, inputCol=column_name, outputCol=out_col_name, seed=RANDOM_SEED)
  model = word2vec.fit(df)
  features = model.transform(df).cache()
  
  return model, features

In [0]:
model, w2v_title_features = extract_w2v_features(clean_title_df, "terms_stemmed", "title_vec")

In [0]:
model, w2v_subjects_features = extract_w2v_features(clean_subjects_df, "terms_stemmed", "subjects_vec")

In [0]:
model, w2v_authors_features = extract_w2v_features(clean_authors_df, "terms", "authors_vec")

In [0]:
# Final dataframe
w2v_title_features = w2v_title_features.select(["id", "title_vec"])
w2v_subjects_features = w2v_subjects_features.select(["id", "subjects_vec"])

In [0]:
w2v_authors_features = w2v_authors_features.select(["id", "authors_vec"])

In [0]:
engineered_books_df = w2v_title_features

In [0]:
engineered_books_df = engineered_books_df.join(w2v_subjects_features, ["id"])

In [0]:
engineered_books_df = engineered_books_df.join(w2v_authors_features, ["id"])

In [0]:
engineered_books_df.show() 

+---+--------------------+--------------------+--------------------+
 id| title_vec| subjects_vec| authors_vec|
+---+--------------------+--------------------+--------------------+
 0|[-0.0233846642076...|[-1.5495021943934...|[-0.0159084685146...|
 1|[-0.0062158408774...|[0.03917153899303...|[-0.0159084685146...|
 2|[-5.9998613627006...|[0.10005942091811...|[-0.0159084685146...|
 3|[-2.4767708964645...|[0.08504896447993...|[-0.0159084685146...|
 4|[-0.0072771161794...|[0.05650274282587...|[-0.0159084685146...|
 5|[-0.0254325654357...|[-0.0152122699510...|[-0.0159084685146...|
 6|[0.03920439042017...|[0.07760200866808...|[0.00200751703232...|
 7|[-9.2597324401140...|[0.03337301659242...|[0.00200751703232...|
 8|[0.03416956868022...|[-1.5495021943934...|[0.00200751703232...|
 9|[0.11026794835925...|[-1.5495021943934...|[-0.0159084685146...|
 10|[0.00400219804474...|[-1.5495021943934...|[-0.0159084685146...|
 11|[0.01653655931692...|[-1.5495021943934...|[0.00200751703232...|
 12|[-0.0275370255112...|[-0.0139251618335...|[0.00200751703232...|
 13|[0.00126327981706...|[-1.5495021943934...|[0.00200751703232...|
 14|[0.01225520230946...|[-1.5495021943934...|[-0.0159084685146...|
 15|[0.0,0.0,0.0,0.0,...|[-1.5495021943934...|[-0.0159084685146...|
 16|[0.02947577663386...|[0.11116931680589...|[0.00200751703232...|
 17|[0.0,0.0,0.0,0.0,...|[-1.5495021943934...|[0.00200751703232...|
 18|[0.0,0.0,0.0,0.0,...|[0.10393913574516...|[0.00200751703232...|
 19|[0.03521133276323...|[0.04840894676744...|[-0.0159084685146...|
+---+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
vec_ass = VectorAssembler(inputCols=["title_vec","subjects_vec","authors_vec"], outputCol="feature", handleInvalid="keep")
feature = vec_ass.transform(engineered_books_df).select(["feature"]).collect()

In [0]:
#TO DO

#create a new dataframe with columns "id" and "feature"


--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-1284119553071802> in <module> 
 ----> 1 my_df = my_df . join ( feature , [ "feature" ] ) 
 2 engineered_books_ass_df = my_df . select ( [ "id" , "feature" ] ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in join (self, other, on, how) 
 1360 on = self . _jseq ( [ ] ) 
 1361 assert isinstance ( how , str ) , "how should be a string" 
 -> 1362 jdf = self . _jdf . join ( other . _jdf , on , how ) 
 1363 return DataFrame ( jdf , self . sql_ctx ) 
 1364 

 AttributeError : 'list' object has no attribute '_jdf'

In [0]:
print(engineered_books_ass_df)

[Row(feature=DenseVector([-0.0062, 0.0067, 0.0008, -0.0054, 0.0023, -0.0013, 0.0059, 0.0067, 0.0039, 0.0048, 0.0046, -0.0051, -0.0075, 0.008, -0.012, -0.0032, -0.0006, -0.0005, 0.0007, -0.0106, 0.0152, -0.0072, 0.0035, 0.0019, 0.0087, 0.0022, 0.0031, 0.0071, 0.0053, -0.0074, -0.0019, 0.0068, 0.0031, 0.0064, 0.0016, -0.003, -0.0057, 0.0074, -0.0053, -0.0026, 0.0022, -0.0023, 0.0002, -0.0003, -0.0058, -0.0022, 0.0071, 0.0051, 0.005, -0.0021, -0.0058, 0.0058, 0.0042, 0.0058, -0.0004, 0.0003, 0.0018, 0.0013, -0.0005, -0.0065, -0.0064, -0.0007, -0.0001, 0.0026, 0.0045, 0.0015, 0.0054, -0.0018, -0.0045, -0.0021, -0.0043, -0.0038, 0.0014, -0.011, -0.0096, 0.0053, -0.0086, -0.0005, -0.0006, 0.0015, -0.0044, -0.0003, 0.0019, 0.0006, 0.0113, -0.0092, -0.0003, 0.0061, -0.0037, 0.0006, -0.0045, -0.0063, 0.0023, 0.0065, 0.0059, 0.0022, 0.0044, 0.0031, 0.0046, 0.0034, -0.002, 0.002, 0.003, 0.0061, -0.0116, 0.0004, -0.0066, 0.0005, 0.0007, 0.0017, 0.0072, -0.0022, 0.0014, 0.0015, 0.0015, 0.0079, 0.0006, 0.0009, 0.0098, 0.0023, 0.0033, -0.0029, 0.0022, 0.0084, -0.0017, -0.0051, 0.003, -0.0049, 0.0012, 0.0098, -0.0, 0.0118, 0.0069, 0.0017, 0.0018, 0.0033, -0.0057, -0.0064, -0.003, 0.005, 0.0053, 0.0077, -0.005, 0.0051, -0.0035, 0.0074, -0.0001, -0.0001, 0.0074, 0.0045, 0.0392, -0.0077, -0.024, -0.0569, -0.0134, 0.0297, -0.0033, 0.2183, -0.0574, 0.1334, 0.09, -0.0536, -0.0292, 0.0614, -0.1033, -0.0278, -0.0606, -0.0428, -0.0879, 0.0113, 0.0631, -0.0047, -0.032, -0.0321, -0.0684, -0.1034, -0.1034, -0.0896, -0.0818, 0.0046, -0.0229, 0.0959, 0.0324, 0.087, -0.0381, -0.0888, 0.0152, 0.0065, -0.0139, -0.0572, -0.0755, -0.0333, -0.07, -0.1185, -0.1006, -0.1268, 0.016, -0.0328, 0.0508, 0.0449, 0.0755, -0.0341, 0.0738, 0.0489, -0.0584, 0.0307, 0.1462, -0.0311, -0.0982, -0.0056, -0.1211, -0.1138, -0.0055, -0.0291, 0.0205, -0.0795, 0.0173, 0.0046, 0.083, 0.098, -0.0129, -0.0763, -0.036, 0.0361, 0.0447, 0.0164, 0.0798, -0.0587, 0.1593, 0.0301, -0.0307, -0.0091, 0.0884, -0.025, 0.1114, 0.0776, -0.0691, -0.0472, 0.0067, 0.0019, -0.0599, 0.0783, -0.0302, 0.0119, -0.0415, 0.0818, 0.0555, 0.0534, -0.042, -0.0073, -0.0445, 0.1049, 0.0679, 0.036, 0.0767, 0.1465, -0.0298, -0.1591, -0.1095, -0.0522, -0.0167, 0.0639, -0.0177, 0.0626, 0.1521, -0.0069, -0.0001, 0.0122, 0.0807, 0.1119, -0.128, -0.0894, 0.0135, 0.116, -0.0329, -0.0056, 0.1036, -0.0558, -0.0554, -0.0624, 0.0208, 0.0171, -0.095, 0.0991, 0.0315, -0.0989, -0.0666, -0.0369, -0.0594, 0.0331, -0.1014, -0.0055, -0.0087, 0.0336, -0.0793, -0.1553, 0.0081, -0.042, 0.0396, 0.0472, -0.0159, 0.2694, -0.3159, -0.1017, -0.1163, -0.2916, -0.306, 0.3171, 0.124, -0.1295, -0.2948, 0.0182, 0.3399, 0.3104, -0.1508, 0.1337, 0.2592, 0.0664, -0.2164, -0.2649, 0.201, -0.2952, 0.1489, -0.2985, -0.0071, 0.0627, -0.1161, 0.248, 0.3366, -0.0578, 0.2645, 0.0749, -0.2983, 0.1257, 0.0156, -0.2452, -0.1359, 0.2509, -0.0988, 0.2712, 0.2911, 0.19, 0.2904, 0.1492, -0.229, 0.2413, 0.0494, 0.0364, 0.3096, -0.1615, -0.1785, 0.0192, 0.132, 0.1445, 0.3105, 0.0441, 0.2258, 0.2331, -0.1573, 0.1434, -0.3271, -0.101, 0.2216, -0.0813, -0.2061, -0.3271, -0.1513, -0.1486, -0.255, 0.3128, -0.0329, -0.0707, 0.1133, 0.1277, 0.2191, 0.1982, 0.1011, -0.1652, 0.3056, 0.1877, -0.3038, -0.1539, 0.341, 0.17, -0.2773, 0.0309, 0.3225, 0.1057, -0.1516, -0.3138, -0.1617, -0.0962, -0.0061, -0.1468, 0.3317, 0.2043, 0.0356, 0.0957, 0.0499, 0.0026, -0.1132, -0.137, -0.0233, 0.1637, -0.0698, 0.2817, 0.3005, 0.2799, -0.2741, -0.0471, 0.1645, 0.1372, 0.2561, -0.1281, 0.2234, 0.1811, -0.1914, -0.2874, -0.1933, 0.2849, -0.2755, 0.1315, 0.0279, -0.0585, 0.0337, 0.0721, -0.0712, 0.1482, -0.2839, 0.1086, 0.1374, 0.2607, -0.1706, -0.203, 0.1431, -0.0073, 0.2514, 0.2586, 0.0351, -0.0092, -0.2298, -0.1636, 0.1824, -0.2452, -0.1572, -0.1544, -0.1544, -0.2985, -0.0389, -0.14])), Row(feature=DenseVector([-0.0006, -0.0002, -0.0011, 0.0017, -0.0003, 0.0007, -0.0012, 0.0006, -0.0001, -0.0017, 0.0009, 0.0013, -0.0006, 0.0002, -0.0004, -0.0019, -0.0013, -0.0011, -0.0, 0.0005, -0.0008, 

##Graph

In [0]:
nodes_df = engineered_books_df.select(["id"])

In [0]:
def dot_prod(x,y):
  my_sum = 0
  i = 0
  for value in x:
    my_sum += x[i] * y[i]
    i += 1
  return my_sum

def my_cosine_similarity(x, y):
  return float(dot_prod(x,y)/(np.linalg.norm(x)*np.linalg.norm(y)))

In [0]:
thr = 0.5
Link = Row("src", "dst")
links = []
for row1 in df:
  for row2 in df:
    x = row1["feature"]
    y = row2["feature"]
    if x != y:
      if my_cosine_similarity(x, y) > thr:
        links.append(Link(row1["id"], row2["id"]))
        links.append(Link(row2["id"], row1["id"]))
      
links_df = spark.createDataFrame(links)

inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!
inserito!


In [0]:
books_graph = GraphFrame(nodes_df, links_df) 

In [0]:
books_CC = books_graph.connectedComponents()

##Clustering

In [0]:
# from Document_Clustering.ipynb  e l'altro del clustering
# k = nCC , kmeans

##Evaluation